Stanislas Deneuville - Emmanuel Ferrandi - Pol Grisart - Marine Médard
# Project of data science :  Face recognition in a video and counting
12/11/2018

## Part III : Use on a video

#### 1. Import the notebook Cleaning_baguette 
_________________________________________________________________________________________________________________________

 I took the code used to import a jupyter notebook on this website : https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html

In [ ]:
import io, os, sys, types

In [ ]:
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [ ]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path


In [ ]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [ ]:
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]


In [ ]:
sys.meta_path.append(NotebookFinder())

#### 2. Apply the result of the model on video
_________________________________________________________________________________________________________________________


In [ ]:
%matplotlib inline

import cv2
from keras.models import model_from_json
import keras
import matplotlib.pyplot as plt
from IPython import display
import matplotlib.patches as patches
import numpy as np
import Cleaning_baguette

__get_model__ : function  that get back a model from the json and f5 files in the current repertory

In [ ]:
def get_model(model_name) :
    # load json and create model
    json_file = open(model_name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(model_name + ".h5")
    print("Loaded model from disk")
    return (loaded_model)

__launch_video__ : function that launch the video gave in parameter with the number of faces on the video showed on the video

In [ ]:
def launch_video(video_name, model) :
    cap = cv2.VideoCapture(video_name)
    ret,frame = cap.read()
    x = 0
    res = 0
    NB_FRAMES = 100
    # The capture last for 100 frames (on my computer this last for about 100s)
    while(x <= NB_FRAMES):
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        # Clean the frame
        image_cleaned = Cleaning_baguette.clean_without_save(frame)
        
        # Get the number of faces of the frame
        res = str(model.predict(image_cleaned.reshape(1, 50, 50, 1)).argmax())

        # Add a rectangle at a corner of the image with the number of faces 
        fig,ax = plt.subplots(figsize=(12.8,7.2))
        font = cv2.FONT_HERSHEY_SIMPLEX
        frame = cv2.rectangle(frame, (10,460), (70,420), (0,255,0), 5)
        cv2.putText(frame, res, (20, 450), font, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
        
        
        # Display the image
        ax.imshow(frame)         
        plt.draw()
        display.clear_output(wait=True)
        display.display(plt.gcf())

        x+=1


    # When everything done, release the capture
    cap.release()
    #cv2.destroyAllWindows()

__launch_wecam__ : the same function than launch_video but with the webcam in real time

In [ ]:
def launch_webcam(model) :
    launch_video(0, model)


In [ ]:
model = get_model("CNN")
launch_webcam(model)